In [62]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [63]:
# !apt-get -q install openjdk-8-jdk-headless -qq > /dev/null
# !wget -qnc https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
# !tar xf spark-3.2.1-bin-hadoop2.7.tgz
# !pip install -q findspark

In [77]:
import os
import findspark
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/home/shane/Co_filter_code/spark-3.2.1-bin-hadoop2.7"
findspark.init()

## [Reference CoFilter With Spark](https://spark.apache.org/docs/2.3.0/ml-collaborative-filtering.html)

In [78]:
import pyspark
from pyspark.sql import SparkSession, Row

from pyspark.sql.types import StructType,StructField,IntegerType,StringType,LongType
from pyspark.sql.functions import col, conv, explode, collect_list, desc, sort_array, rank
from pyspark.ml.recommendation import ALS
# from pyspark.ml.evaluation import RegressionMetrics, RankingMetrics

In [79]:
userCount = "all"

ss = SparkSession.builder.appName('Colab').getOrCreate()
inputParsed = ss.read.options(header=True, inferSchema='True',delimiter=',').csv(f"./co_filter/sampled_{userCount}_user_all_purchase.csv").rdd
ratingRDD = inputParsed.map(lambda p: Row(customerId=int(p[1]), articleId=int(p[2]), count=int(p[3])))
ratingDf = ss.createDataFrame(ratingRDD)
ratingMean = ratingDf.groupby('customerId').mean("count")
ratingDf = ratingDf.join(ratingMean, on="customerId", how="left")
ratingDf = ratingDf.withColumn("countNorm", ratingDf["count"] - ratingDf["avg(count)"])
# ratingDf.show(5)

import pickle
def pickle_dump(path, d):
    with open(path, 'wb+') as f:
        pickle.dump(d, f)
pickle_dump(f"./co_filter/ratings_averaged_{userCount}_user.pkl", ratingDf.collect())


22/04/12 05:50:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: _c0, customer_id, article_id, rating
Expected: _c0 but found: 
CSV file: file:///home/shane/Co_filter_code/co_filter/sampled_1000_user_all_purchase.csv
22/04/12 05:50:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: _c0, customer_id, article_id, rating
Expected: _c0 but found: 
CSV file: file:///home/shane/Co_filter_code/co_filter/sampled_1000_user_all_purchase.csv
22/04/12 05:50:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: _c0, customer_id, article_id, rating
Expected: _c0 but found: 
CSV file: file:///home/shane/Co_filter_code/co_filter/sampled_1000_user_all_purchase.csv


In [67]:



# predictions = model.transform(test)
# evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",
#                                 predictionCol="prediction")
# rmse = evaluator.evaluate(predictions)

# userRecs = model.recommendForAllUsers(3)
# evaluator = RegressionMetrics()
# rmse = evaluator.evaluate(predictions)
# print("Root-mean-square error = " + str(rmse))
# userRecs.show(3)

In [68]:
# from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
# from pyspark.ml.evaluation import RegressionEvaluator

# param_grid = ParamGridBuilder() \
#             .addGrid(als.rank, [10, 50, 100, 150]) \
#             .addGrid(als.regParam, [.01, .05, .1, .15]) \
#             .build()

# # Define evaluator as RMSE and print length of evaluator
# evaluator = RegressionEvaluator(
#            metricName="rmse", 
#            labelCol="countNorm", 
#            predictionCol="prediction") 
# print ("Num models to be tested: ", len(param_grid))


# # Build cross validation using CrossValidator
# cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# #Fit cross validator to the 'train' dataset
# model = cv.fit(train)
# #Extract best model from the cv model above
# best_model = model.bestModel
# # View the predictions
# test_predictions = best_model.transform(test)
# RMSE = evaluator.evaluate(test_predictions)
# print(RMSE)

# print("**Best Model**")
# # Print "Rank"
# print("  Rank:", best_model._java_obj.parent().getRank())
# # Print "MaxIter"
# print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
# # Print "RegParam"
# print("  RegParam:", best_model._java_obj.parent().getRegParam())

In [80]:
(train, test) = ratingDf.randomSplit([0.8, 0.2])
als = ALS(maxIter=5, regParam=0.01, userCol="customerId", itemCol="articleId", ratingCol="countNorm",
          coldStartStrategy="drop")
model = als.fit(train)

# Generate n Recommendations for all users
recommendations = model.recommendForAllUsers(12)
pickle_dump(f"./co_filter/recommendations_{userCount}_user.pkl", recommendations.collect())

nrecommendations = recommendations.withColumn("rec_exp", explode("recommendations"))\
    .select('customerId', col("rec_exp.articleId"), col("rec_exp.rating"))



from pyspark.sql.window import Window
w = Window.partitionBy(nrecommendations.customerId).orderBy(nrecommendations.rating)
rec_ranked = nrecommendations.withColumn("rank", rank().over(w))
rec_list = rec_ranked.groupBy("customerId") .agg(collect_list("articleId").alias("articleList"))
# rec_list.show(5)





22/04/12 05:50:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: _c0, customer_id, article_id, rating
Expected: _c0 but found: 
CSV file: file:///home/shane/Co_filter_code/co_filter/sampled_1000_user_all_purchase.csv
22/04/12 05:50:13 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: _c0, customer_id, article_id, rating
Expected: _c0 but found: 
CSV file: file:///home/shane/Co_filter_code/co_filter/sampled_1000_user_all_purchase.csv
22/04/12 05:50:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: _c0, customer_id, article_id, rating
Expected: _c0 but found: 
CSV file: file:///home/shane/Co_filter_code/co_filter/sampled_1000_user_all_purchase.csv
22/04/12 05:50:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , customer_id, article_id, rating
 Schema: 

In [81]:
import pandas
import pickle
def pickle_load(path):
    with open(path, 'rb') as f:
        return pickle.load(f)


rec_dict = rec_list.toPandas()
customerIdDecode = pickle_load(f"./co_filter/customer_id_decoding_{userCount}_user.pkl")
articleIdDecode = pickle_load(f"./co_filter/article_id_decoding_{userCount}_user.pkl")

rec_dict.replace({"customerId": customerIdDecode}, inplace=True)
rec_dict["articleList"] = rec_dict["articleList"].apply(lambda x: list(itemgetter(*x)(articleIdDecode)))

def write_formatted(df, fp):
    df["stringTemp"] = df["articleList"].apply(lambda x: str(x)[1:-1].replace(',', ""))
    df[["customerId", "stringTemp"]].to_csv(fp, header=["customer_id","prediction"], index=None, sep=',', mode='w+')
    
write_formatted(rec_dict, f"./result{userCount}user.csv")
